# Amazon Bedrock モデルを使用した LangGraph エージェントの Amazon Bedrock AgentCore Runtime でのホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用して既存のエージェントをホストする方法を学びます。

LangGraph と Amazon Bedrock モデルの例に焦点を当てます。Strands エージェントと Amazon Bedrock モデルについては[こちら](../01-strands-with-bedrock-model)を、Strands エージェントと OpenAI モデルについては[こちら](../03-strands-with-openai-model)を参照してください。

### チュートリアルの詳細

| 情報                | 詳細                                                                         |
|:--------------------|:-----------------------------------------------------------------------------|
| チュートリアルの種類 | 会話型                                                                       |
| エージェントの種類   | 単一                                                                         |
| エージェントフレームワーク | LangGraph                                                                    |
| LLM モデル          | Anthropic Claude Sonnet 3                                                    |
| チュートリアルコンポーネント | AgentCore Runtime でのエージェントのホスティング。LangGraph と Amazon Bedrock モデルの使用 |
| チュートリアルの分野 | 分野横断                                                                     |
| 例の複雑さ          | 簡単                                                                         |
| 使用した SDK        | Amazon BedrockAgentCore Python SDK と boto3                                  |

### チュートリアルのアーキテクチャ

このチュートリアルでは、既存のエージェントを AgentCore ランタイムにデプロイする方法を説明します。

デモンストレーション目的で、Amazon Bedrock モデルを使用した LangGraph エージェントを使用します。

例では、`get_weather` と `get_time` の 2 つのツールを持つ非常に単純なエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントのホスティング
* Amazon Bedrock モデルの使用
* LangGraph の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です:

* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker の実行

In [ ]:
#!uv add -r requirements.txt --active

In [ ]:
import os
# os.environ['AWS_PROFILE'] = 'your_profile_name'

# べき等ではないため完全に一から再実行する場合は `rm .bedrock_agentcore.yaml` を実行してください。

## エージェントの作成とローカルでの実験

AgentCore Runtimeにエージェントをデプロイする前に、実験目的でローカルでエージェントを開発して実行しましょう。

本番のエージェントアプリケーションでは、エージェントの作成プロセスとエージェントの呼び出しプロセスを分離する必要があります。AgentCore Runtimeでは、エージェントの呼び出し部分に `@app.entrypoint` デコレータを付けて、ランタイムのエントリポイントとします。まずは、実験段階でそれぞれのエージェントがどのように開発されるかを見ていきましょう。

ここでのアーキテクチャは次のようになります。

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import argparse
import json
import operator
import math
import sys

# 計算機ツールを作成
@tool
def calculator(expression: str) -> str:
    """数式の計算結果を返します。
    
    Args:
        expression: 文字列形式の数式 (例: "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        計算結果を文字列として返します
    """
    try:
        # 式で使用可能な安全な関数を定義
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # 数学関数
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # 基本演算子（明示的な使用のため）
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # 式を安全に評価
        result = eval(expression, safe_dict)
        return str(result)
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# カスタム天気ツールを作成
@tool
def weather():
    """Get weather"""  # ダミー実装
    return "sunny"

# 手動でLangGraphを構築してエージェントを定義
def create_agent():
    """LangGraphエージェントを作成・設定"""
    from langchain_aws import ChatBedrock
    
    # LLMを初期化（必要に応じてモデルとパラメータを調整）
    llm = ChatBedrock(
        model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        model_kwargs={"temperature": 0.1}
    )
    
    # ツールをLLMにバインド
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # システムメッセージ
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # チャットボットノードを定義
    def chatbot(state: MessagesState):
        # システムメッセージがまだ存在しない場合は追加
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # グラフを作成
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # エッジを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # エントリーポイントを設定
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイル
    return graph_builder.compile()

# エージェントを初期化
agent = create_agent()

def langgraph_bedrock(payload):
    """ペイロードでエージェントを呼び出し"""
    user_input = payload.get("prompt")
    
    # LangGraphが期待する形式で入力を作成
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # 最終メッセージの内容を抽出
    return response["messages"][-1].content

# Jupyter環境で実行されているかどうかを確認する関数
def is_jupyter():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':  # Jupyter notebook or qtconsole
            return True
        elif shell == 'TerminalInteractiveShell':  # Terminal IPython
            return False
        else:
            return False
    except NameError:  # 通常のPythonインタプリタ
        return False

# メイン処理
if __name__ == "__main__":
    # Jupyter環境で実行されている場合のサンプル使用法を提供
    if is_jupyter():
        # Jupyter環境では、以下のように直接関数を呼び出すことができます
        sample_payload = {"prompt": "2+2はいくつですか？"}
        response = langgraph_bedrock(sample_payload)
    else:
        # コマンドライン引数からの実行（元の動作）
        parser = argparse.ArgumentParser()
        parser.add_argument("payload", type=str)
        args = parser.parse_args()
        
        response = langgraph_bedrock(json.loads(args.payload))
    print(response)


#### ローカルエージェントの呼び出し

In [ ]:
!python langgraph_bedrock.py '{"prompt": "今の天気は?"}'

## AgentCore Runtimeにエージェントを展開する準備

さて、私たちのエージェントをAgentCore Runtimeに展開しましょう。そのためには、以下の手順が必要です。

* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポートする
* `app = BedrockAgentCoreApp()` でコード内でアプリを初期化する
* 呼び出し関数に `@app.entrypoint` デコレータを付ける
* `app.run()` でAgentCoreRuntimeがエージェントの実行を制御する

### Amazon Bedrock モデルを使った LangGraph
Amazon Bedrock モデルを使った LangGraph から始めましょう。異なるフレームワークやモデルを使った他の例は、親ディレクトリにあります。

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

# 計算機ツールを作成
@tool
def calculator(expression: str) -> str:
    """Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # 式で使用可能な安全な関数を定義
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # 数学関数
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # 基本演算子（明示的な使用のため）
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # 式を安全に評価
        result = eval(expression, safe_dict)
        return str(result)
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# カスタム天気ツールを作成
@tool
def weather():
    """Get weather"""  # ダミー実装
    return "sunny"

# 手動でLangGraphを構築してエージェントを定義
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # LLMを初期化（必要に応じてモデルとパラメータを調整）
    llm = ChatBedrock(
        model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        model_kwargs={"temperature": 0.1}
    )
    
    # ツールをLLMにバインド
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # システムメッセージ
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # チャットボットノードを定義
    def chatbot(state: MessagesState):
        # システムメッセージがまだ存在しない場合は追加
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # グラフを作成
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # エッジを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # エントリーポイントを設定
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイル
    return graph_builder.compile()

# エージェントを初期化
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    """Invoke the agent with a payload"""
    user_input = payload.get("prompt")
    
    # LangGraphが期待する形式で入力を作成
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # 最終メッセージの内容を抽出
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

## 裏側で何が起こっているのか?

`BedrockAgentCoreApp` を使用すると、自動的に以下のことが行われます。

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するために必要な `/invocations` エンドポイントを実装
* ヘルスチェック (非同期エージェントにとって非常に重要) のための `/ping` エンドポイントを実装
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS 標準に従ったエラー処理を管理

## AgentCore ランタイムにエージェントをデプロイする

`CreateAgentRuntime` 操作は包括的な構成オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントと通信する方法を制御するために、プロトコル設定 (HTTP、MCP) と認証メカニズムを構成することもできます。

**注:** オペレーションのベストプラクティスは、コードをコンテナにパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore ランタイムにデプロイします。

### ランタイムロールの作成

始める前に、AgentCore Runtimeの IAM ロールを作成しましょう。あらかじめ開発された utils 関数を使用して行います。

In [ ]:
import sys
import os
import json
import boto3

# 現在のノートブックのディレクトリを取得
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# utils.py の場所に移動
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# sys.path に追加
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="langgraph_bedrock"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

### AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、AgentCore Runtime デプロイをエントリポイント、作成した実行ロール、および要件ファイルで設定します。また、スターターキットを設定して、起動時に Amazon ECR リポジトリを自動的に作成するようにします。

設定ステップでは、アプリケーションコードに基づいて Dockerfile が生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

### AgentCore Runtimeへのエージェントの起動

Dockerfileを作成したので、次はAgentCore Runtimeにエージェントを起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### AgentCore Runtimeのステータスを確認する
AgentCore Runtimeをデプロイしたので、次はそのデプロイ状況を確認しましょう

日本語訳:

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtimeの呼び出し

最後に、ペイロードを使って AgentCore Runtime を呼び出すことができます。

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "2+2は?"})
invoke_response

### 呼び出し結果の処理

アプリケーションに組み込むために、呼び出し結果を処理することができます。

日本語訳:

### 呼び出し結果の処理

私たちは今や、アプリケーションに組み込むために、呼び出し結果を処理することができます。

In [ ]:
from IPython.display import Markdown, display
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### AgentCore Runtimeをboto3で呼び出す

AgentCore Runtimeが作成されたので、任意のAWS SDKでそれを呼び出すことができます。例えば、boto3の `invoke_agent_runtime` メソッドを使用できます。

日本語訳:
### AgentCore Runtimeを boto3 で呼び出す

AgentCore Runtime が作成されたので、任意の AWS SDK でそれを呼び出すことができます。例えば、boto3 の `invoke_agent_runtime` メソッドを使用できます。

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "今の天気は?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## クリーンアップ (オプション)

作成した AgentCore Runtime をクリーンアップしましょう。

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# IAMロールの名前を取得
agentcore_role_name = agentcore_iam_role['Role']['RoleName']

policies = iam_client.list_role_policies(
    RoleName=agentcore_role_name,
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_role_name,
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_role_name
)

# Congratulations!